# Val and Marta's Final Project
## CSE360/460

### Frisbee-Catching Robot

In [1]:
import sim
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [9]:
class robot():
    
    def __init__(self, frame_name, motor_names=[], client_id=0):  
        # If there is an existing connection
        if client_id:
                self.client_id = client_id
        else:
            self.client_id = self.open_connection()
            
        self.motors = self._get_handlers(motor_names) 
        
        # Robot frame
        self.frame =  self._get_handler(frame_name)
            
        
    def open_connection(self):
        sim.simxFinish(-1)  # just in case, close all opened connections
        self.client_id = sim.simxStart('127.0.0.1', 19999, True, True, 5000, 5)  # Connect to CoppeliaSim 
        
        if clientID != -1:
            print('Robot connected')
        else:
            print('Connection failed')
        return clientID
        
    def close_connection(self):    
        sim.simxGetPingTime(self.client_id)  # Before closing the connection to CoppeliaSim, make sure that the last command sent out had time to arrive.
        sim.simxFinish(self.client_id)  # Now close the connection to CoppeliaSim:
        print('Connection closed')
    
    def isConnected(self):
        c,result = sim.simxGetPingTime(self.client_id)
        # Return true if the robot is connected
        return result > 0         
        
    def _get_handler(self, name):
        err_code, handler = sim.simxGetObjectHandle(self.client_id, name, sim.simx_opmode_blocking)
        return handler
    
    def _get_handlers(self, names):
        handlers = []
        for name in names:
            handler = self._get_handler(name)
            handlers.append(handler)
        
        return handlers

    def send_motor_velocities(self, vels):
        for motor, vel in zip(self.motors, vels):
            err_code = sim.simxSetJointTargetVelocity(self.client_id, 
                                                      motor, vel, sim.simx_opmode_streaming)      
            
    def set_position(self, position, relative_object=-1):
        if relative_object != -1:
            relative_object = self._get_handler(relative_object)        
        sim.simxSetObjectPosition(clientID, self.frame, relative_object, position, sim.simx_opmode_oneshot)
        
    def simtime(self):
        return sim.simxGetLastCmdTime(self.client_id)
    
    def get_position(self, relative_object=-1):
        # Get position relative to an object, -1 for global frame
        if relative_object != -1:
            relative_object = self._get_handler(relative_object)
        res, position = sim.simxGetObjectPosition(self.client_id, self.frame, relative_object, sim.simx_opmode_blocking)        
        return array(position)
    
    def get_object_position(self, object_name):
        # Get Object position in the world frame
        err_code, object_h = sim.simxGetObjectHandle(self.client_id, object_name, sim.simx_opmode_blocking)
        res, position = sim.simxGetObjectPosition(self.client_id, object_h, -1, sim.simx_opmode_blocking)
        return array(position)
    
    def get_object_relative_position(self, object_name):        
        # Get Object position in the robot frame
        err_code, object_h = sim.simxGetObjectHandle(self.client_id, object_name, sim.simx_opmode_blocking)
        res, position = sim.simxGetObjectPosition(self.client_id, object_h, self.frame, sim.simx_opmode_blocking)
        return array(position)
    

In [10]:
# For finding the coefficients for splines, in order to move the frisbee
def point_to_point_traj(x1, x2, v1, v2, delta_t):
    # the following is finding each coefficient for the spline
    a0 = x1
    a1 = v1
    a2 = (3*x2 - 3*x1 - 2*v1*delta_t - v2 * delta_t) / (delta_t**2)
    a3 = (2*x1 + (v1 + v2) * delta_t  - 2 * x2) / (delta_t**3)
    return a0, a1, a2, a3


In [4]:
import sim as vrep
import time
import cv2
import numpy as np
from random import * # for random numbers
clientID = sim.simxStart('127.0.0.1', 19999, True, True, 5000, 5)  # Connect to CoppeliaSim

# Move the frisbee
disc = robot('Sphere')

# Create trajectory for disc 
random_num = 0
#randint(0, 2)  # pick a number between 0 and 2

# TODO: Fix trajectories to be what we want (can be done anytime)
if random_num == 0:
    x_i, x_f, vx_i, vx_f = 1.2, 0.5, 0.1, 0.1
    y_i, y_f, vy_i, vy_f = -4.4, 5.5, 0.2, 0.2
    z_i, z_f, vz_i, vz_f = 3, 1.5, -0.5, -0.5
elif random_num == 1:
    x_i, x_f, vx_i, vx_f = 0, 0, 0, 0
    y_i, y_f, vy_i, vy_f = 0, 0, 0, 0
    z_i, z_f, vz_i, vz_f = 0, 0, 0, 0
elif random_num == 2:
    x_i, x_f, vx_i, vx_f = 0, 0, 0, 0
    y_i, y_f, vy_i, vy_f = 0, 0, 0, 0
    z_i, z_f, vz_i, vz_f = 0, 0, 0, 0
    
a0x, a1x, a2x, a3x = point_to_point_traj(x_i, x_f, vx_i, vx_f, 20)
a0y, a1y, a2y, a3y = point_to_point_traj(y_i, y_f, vy_i, vy_f, 20)
a0z, a1z, a2z, a3z = point_to_point_traj(z_i, z_f, vz_i, vz_f, 20)

# Time interval
time_steps = linspace(0, 20, 200)

res, v1 = vrep.simxGetObjectHandle(clientID, 'Vision_sensor', vrep.simx_opmode_oneshot_wait)
print ('Getting first image')
err, resolution, image = vrep.simxGetVisionSensorImage(clientID, v1, 0, vrep.simx_opmode_streaming)
orientations = []
images = []

for t in time_steps:
    tx, ty, tz = a0x + a1x * t + a2x * t**2 + a3x * t**3, a0y + a1y * t + a2y * t**2 + a3y * t**3, a0z + a1z * t + a2z * t**2 + a3z * t**3
    disc.set_position([tx, ty, tz])
    
    err, resolution, image = vrep.simxGetVisionSensorImage(clientID, v1, 0, vrep.simx_opmode_buffer)
    orientation = vrep.simxGetObjectOrientation(clientID, v1, -1, vrep.simx_opmode_blocking)[1][1]
    orientations.append(orientation)

    img = np.array(image,dtype=np.uint8)
    img.resize([resolution[1],resolution[0],3])
    images.append(img)
    imshow(img)
    time.sleep(0.1)


NameError: name 'sim' is not defined

In [2]:


if clientID!=-1:
    # Handler for the camera
    res, v1 = vrep.simxGetObjectHandle(clientID, 'Vision_sensor', vrep.simx_opmode_oneshot_wait)
    print ('Getting first image')
    err, resolution, image = vrep.simxGetVisionSensorImage(clientID, v1, 0, vrep.simx_opmode_streaming)
    orientations = []
    images = []
    err, resolution, image = vrep.simxGetVisionSensorImage(clientID, v1, 0, vrep.simx_opmode_buffer)
    orientation = vrep.simxGetObjectOrientation(clientID, v1, -1, vrep.simx_opmode_blocking)[1][1]
    orientations.append(orientation)
    if err == vrep.simx_return_ok:
        print ("image %d OK!!!"%i)
        img = np.array(image,dtype=np.uint8)
        img.resize([resolution[1],resolution[0],3])
        images.append(img)
        imshow(img)
        i+=1
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

            
        elif err == vrep.simx_return_novalue_flag:
            print ("no image yet")
            pass
        else:
            print (err)
            time.sleep(1)
        
else:
    print ("Failed to connect to remote API Server")
    vrep.simxFinish(clientID)

SyntaxError: invalid syntax (<ipython-input-2-ffe674bcd107>, line 21)